# Hello OpenAI Function Calling

In [1]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# From OpenAI's Function Calling documentation: https://platform.openai.com/docs/guides/gpt/function-calling

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation(question):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": question}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default and lets the model decide whether function should be called. Can also set this to "none" (never) or include the name of the function to force it to call it: {"name": "get_current_weather"}
        # function_call = {"name": "get_current_weather"}
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response
    else:
        return response

## A question that's relevant to the function

In [6]:
print(run_conversation("What's the weather like in Boston?"))

{
  "id": "chatcmpl-8CbFf59DYKeQHyHepeS7ci5rde9Lg",
  "object": "chat.completion",
  "created": 1698013823,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is 72 degrees Fahrenheit. The forecast for today is sunny and windy."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 22,
    "total_tokens": 94
  }
}


## A question that's not relevant to the function

In [7]:
print(run_conversation("Why is the sky blue?"))

{
  "id": "chatcmpl-8CbFp5wEP6JLJpdEyC1S7buawGYxJ",
  "object": "chat.completion",
  "created": 1698013833,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight reaches Earth's atmosphere, it encounters molecules and tiny particles like dust and water droplets. These particles scatter the sunlight in all directions. \n\nHowever, the shorter wavelengths of light (blue and violet) are scattered more than the longer wavelengths (red, orange, and yellow). This scattering process causes the blue light to be scattered in all directions, making the sky appear blue. Additionally, our eyes are more sensitive to blue light, which further enhances the perception of a blue sky.\n\nIt's important to keep in mind that the color of the sky can vary based on various factors such as the time of day, weather conditions, and pollutio